# Multiset Partitioning (Facebook)
##### *Algorithm & Data Structures*

Given a multiset of integers, return whether it can be partitioned into two subsets whose sums are the same.

For example, given the multiset $\{15,\ 5,\ 20,\ 10,\ 35,\ 15,\ 10\}$, it would return `True` since we can split it up into $\{15,\ 5,\ 10,\ 15,\ 10\}$ and $\{20,\ 35\}$, which both add up to 55.

You can assume that there are no spaces at the ends of the string and that there is exactly one space between each word.

### Solution
The naive, brute force solution would be to try every combination of two subsets and check their sums. We could do this by trying to generate each subset of our input set, and then checking the sum of that subset with the sum of everything not in the subset.

To speed this up, notice that we really only need to find a subset that adds up to half of the total sum of all the integers. This is because of the pigeonhole principle: if one subset adds up to half of the sum, then the rest of the sum must be made up of the rest of the set.

So, we can generate the powerset of our original set and check if any of them sum to `k / 2`, where `k` is the sum of the original set. We know immediately that if `k` is odd, then we can't partition the sets, so we can immediately return `False`.

Thus, we have the following code:

In [ ]:
def power_set(s):
    if not s:
        return [[]]
    result = power_set(s[1:])
    return result + [subset + [s[0]] for subset in result]

def partition(s):
    k = sum(s)
    if k % 2 != 0:
        return False

    for subset in power_set(s):
        if sum(subset) == k / 2:
            return True

    return False

This will run in $O(n * 2^n)$ time though, since we must generate every subset and sum them up. Can we make this any faster?

Notice that we've reduced the problem into finding a subset of integers that add up to `k / 2`, which is exactly the same as finding a subset of integers that sum up to `k` (a different `k`).

We solved that problem in the past by creating a matrix of size `len(nums) + 1` by `k + 1`, and then using dynamic programming to fill up the matrix. We can do something similar here, except we'll use our `k / 2` as our target.

Each entry `A[i][j]` in our matrix will represent whether we can make the integer `i` with the elements of our set from `0` to `j`. So we'll do the following:
1. Create a matrix of size `k + 1` by `len(s) + 1` of booleans (all initialized to `False`).
2. Initialize the top row to `True`, since we can make `0` with anything (by not picking anything)
3. Iterate over the matrix from top-to-bottom, then left-to-right:
    1. At each index `A[i][j]`, look at `A[i][j - 1]` or `A[i - last][j - 1]` and set to `True` if any are true.
4. Return the value at the bottom-right of the matrix.

Thus, we have the following code:

In [2]:
def partition(s):
    # Step 1
    k = sum(s)
    if k % 2 != 0:
        return False
    half_k = k // 2

    A = [[False for _ in range(len(s) + 1)] for _ in range(half_k + 1)]

    # Step 2
    for j in range(len(s) + 1):
        A[0][j] = True

    # Step 3
    for i in range(1, half_k + 1):
        for j in range(1, len(s) + 1):
            using_last = i - s[j - 1]
            if using_last >= 0:
                A[i][j] = A[i][j - 1] or A[using_last][j - 1]
            else:
                A[i][j] = A[i][j - 1]

    # Step 4
    return A[-1][-1]

This will take $O(kn)$ time and space, just like in the knapsack problem covered in one of our prep sessions last year.

Let's step through this partition function on the multiset $\{1,\ 2,\ 3\}$.

In [7]:
multiset = [1, 1, 2]
partition(multiset)

True

In [8]:
# After step 1, we have
s = [1, 1, 2]
k = 6
half_k = 3
A = [
    [False, False, False, False],
    [False, False, False, False],
    [False, False, False, False],
    [False, False, False, False]
]

# The dimensions of our array are: len(multiset) + 1 by half_k + 1 (4 x 4)


# After step 2, we have
A = [
    [True, True, True, True],
    [False, False, False, False],
    [False, False, False, False],
    [False, False, False, False]
]

Now we iterate over each element (except the first) of each row in the matrix, starting with row 2 (index = 1).

In [10]:
# i = 1, j = 1
# using_last = i - s[j - 1]
# ::: using_last = 1 - s[0] = 1 - 1 = 0
# Since using_last >= 0:
# A[i][j] = A[i][j - 1] or A[using_last][j - 1]
# ::: A[1][1] = A[1][0] or A[0][0] = False or True = True
A = [
    [True, True, True, True],
    [False, True, False, False],
    [False, False, False, False],
    [False, False, False, False]
]

# i = 1, j = 2
# using_last = i - s[j - 1]
# ::: using_last = 1 - s[1] = 1 - 1 = 0
# Since using_last >= 0:
# A[i][j] = A[i][j - 1] or A[using_last][j - 1]
# ::: A[1][2] = A[1][1] or A[0][1] = True or True = True
A = [
    [True, True, True, True],
    [False, True, True, False],
    [False, False, False, False],
    [False, False, False, False]
]

# i = 1, j = 3
# using_last = i - s[j - 1]
# ::: using_last = 1 - s[2] = 1 - 2 = -1
# Since using_last < 0:
# A[i][j] = A[i][j - 1]
# ::: A[1][2] = A[1][2] = True
A = [
    [True, True, True, True],
    [False, True, True, True],
    [False, False, False, False],
    [False, False, False, False]
]

# i = 2, j = 1
# using_last = i - s[j - 1]
# ::: using_last = 2 - s[0] = 2 - 1 = 1
# Since using_last >= 0:
# A[i][j] = A[i][j - 1] or A[using_last][j - 1]
# ::: A[2][1] = A[2][0] or A[1][0] = False or False = False
A = [
    [True, True, True, True],
    [False, True, True, True],
    [False, False, False, False],
    [False, False, False, False]
]

# i = 2, j = 2
# using_last = i - s[j - 1]
# ::: using_last = 2 - s[1] = 2 - 1 = 1
# Since using_last >= 0:
# A[i][j] = A[i][j - 1] or A[using_last][j - 1]
# ::: A[2][2] = A[2][1] or A[1][1] = False or True = True
A = [
    [True, True, True, True],
    [False, True, True, True],
    [False, False, True, False],
    [False, False, False, False]
]

# i = 2, j = 3
# using_last = i - s[j - 1]
# ::: using_last = 2 - s[2] = 2 - 2 = 0
# Since using_last >= 0:
# A[i][j] = A[i][j - 1] or A[using_last][j - 1]
# ::: A[2][3] = A[2][2] or A[0][2] = True or True = True
A = [
    [True, True, True, True],
    [False, True, True, True],
    [False, False, True, True],
    [False, False, False, False]
]

# i = 3, j = 1
# using_last = i - s[j - 1]
# ::: using_last = 3 - s[0] = 3 - 1 = 2
# Since using_last >= 0:
# A[i][j] = A[i][j - 1] or A[using_last][j - 1]
# ::: A[3][1] = A[3][0] or A[2][0] = False or False = False
A = [
    [True, True, True, True],
    [False, True, True, True],
    [False, False, True, True],
    [False, False, False, False]
]

# i = 3, j = 2
# using_last = i - s[j - 1]
# ::: using_last = 3 - s[1] = 3 - 1 = 2
# Since using_last >= 0:
# A[i][j] = A[i][j - 1] or A[using_last][j - 1]
# ::: A[3][2] = A[3][1] or A[2][1] = False or False = False
A = [
    [True, True, True, True],
    [False, True, True, True],
    [False, False, True, True],
    [False, False, False, False]
]

# i = 3, j = 3
# using_last = i - s[j - 1]
# ::: using_last = 3 - s[2] = 3 - 2 = 1
# Since using_last >= 0:
# A[i][j] = A[i][j - 1] or A[using_last][j - 1]
# ::: A[3][3] = A[3][2] or A[1][2] = False or True = True
A = [
    [True, True, True, True],
    [False, True, True, True],
    [False, False, True, True],
    [False, False, False, True]
]

# Finally, in step 4 we return A[-1][-1]
A[-1][-1]

True

This is an example of dynamic programming! It is a concept which is relatively difficult to wrap one's head around at first, but I think the following excerpt from [this GeeksforGeeks article](https://www.geeksforgeeks.org/dynamic-programming/) sums it up well:

> Dynamic Programming is mainly an optimization over plain [recursion](https://www.geeksforgeeks.org/recursion/). Wherever we see a recursive solution that has repeated calls for same inputs, we can optimize it using Dynamic Programming. The idea is to simply store the results of subproblems, so that we do not have to re-compute them when needed later. This simple optimization reduces time complexities from exponential to polynomial. For example, if we write a simple recursive solution for [Fibonacci Numbers](https://www.geeksforgeeks.org/program-for-nth-fibonacci-number/), we get exponential time complexity and if we optimize it by storing solutions of subproblems, time complexity reduces to linear.